# Data Sciense Flow

###1. Target formulation

###2. Text cleaning

###3. Model building

###4. Results using

# Text Categorization

# <img src="classify.png", style="max-width:90%; width: 80%">

# LDA

<img src="LDA.png", style="max-width:90%; width: 80%">

In [10]:

import gensim
from sklearn.datasets import fetch_20newsgroups
import logging
from collections import defaultdict

from nltk.corpus import stopwords
stopwords = stopwords.words("english")
from pattern.en import lemma

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


corpus_savepath = 'gensimldatest.mm'


#basic preprocessing and lemmatization 

texts = fetch_20newsgroups(subset='train').data
texts = [unicode(d.lower()) for d in texts]
texts = ["".join((char if char.isalpha() else " ") for char in text).split() for text in texts]
texts = [[word for word in text[:1000] if word not in stopwords] for text in texts]

#creating frequency dictionary for tokens in text
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

#removing very infrequent and very frequent tokens in corpus
texts = [[token for token in text if (frequency[token] > 10 and len(token) > 2 and frequency[token] < len(texts)*0.2)] for text in texts]

#creating an LDA model
dictionary = gensim.corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
gensim.corpora.MmCorpus.serialize(corpus_savepath, corpus)
modelled_corpus = gensim.corpora.MmCorpus(corpus_savepath)
lda1 = gensim.models.ldamodel.LdaModel(modelled_corpus, num_topics=10, update_every=100, passes=20, id2word=dictionary, alpha='auto', eval_every=5)

#returning the resulting topics
lda1.show_topics(num_topics=10, num_words=30, formatted=True)

# Results analysis

In [145]:
news = "Indianes women hockey could never recover from three early setbacks and ended up losing the Hawkes \
Bay Cup quarterfinal 1-3 to Japan, today. "

texts = [unicode(d.lower()) for d in new.split()]
text1 = dictionary.doc2bow(texts)
sorted(lda[text1], key = lambda x: x[1], reverse = True)

[(6, 0.53758295629888098),
 (2, 0.18588873448018561),
 (8, 0.15471837618089992),
 (1, 0.10088456902754719)]

In [143]:
new = " The bible is not the WORD OF GOD. And if we believe it as such, then we have made the bible into an idol. \
The Bible makes the claim \
of being the inspired word of God, God's message to humanity."

texts = [unicode(d.lower()) for d in new.split()]
text1 = dictionary.doc2bow(texts)
sorted(lda[text1], key = lambda x: x[1], reverse = True)

[(7, 0.48670069241050118),
 (0, 0.24817498942542299),
 (2, 0.13441145401025376),
 (5, 0.10996778497322443)]

In [149]:
new = "Having had windows live mail working satisfactorily for some time "

texts = [unicode(d.lower()) for d in new.split()]
text1 = dictionary.doc2bow(texts)
lda[text1]
sorted(lda[text1], key = lambda x: x[1], reverse = True)

[(6, 0.71185683311735759),
 (4, 0.21705128852495115),
 (9, 0.013788021255459389),
 (1, 0.011798482860266721)]

In [155]:
text1 = dictionary.doc2bow(["government"])
lda[text1]
sorted(lda[text1], key = lambda x: x[1], reverse = True)

[(9, 0.77765272176558953),
 (1, 0.03776186665686139),
 (8, 0.031314062631686457),
 (0, 0.029998017363570186),
 (5, 0.025090172634431366),
 (7, 0.022590981930248007),
 (6, 0.022449774249397886),
 (3, 0.020767332524668225),
 (4, 0.016492515697128261),
 (2, 0.015882554546418556)]

# LDA2Vec

<img src="lda2.jpg", style="max-width:90%; width: 60%">

In [3]:
from lda2vec import preprocess, LDA2Vec, Corpus
from sklearn.datasets import fetch_20newsgroups
from chainer import serializers
from chainer import cuda
import numpy as np
import os.path
import logging


# Optional: moving the model to the GPU makes it ~10x faster
# set to False if you're having problems with Chainer and CUDA
gpu = cuda.available

logging.basicConfig()

from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

corpus_savepath = 'gensimldatest.mm'

#basic preprocessing and lemmatization 
texts = fetch_20newsgroups(subset='train').data
texts = [unicode(d.lower()) for d in texts]
texts = ["".join((char if char.isalpha() else " ") for char in text).split() for text in texts]
texts = [[word for word in text[:2000] if word not in cachedStopWords] for text in texts]

texts = [unicode(d) for d in texts]



# Preprocess data
max_length = 1000   # Limit of 1k words per document
tokens, vocab = preprocess.tokenize(texts, max_length, tag=False,
                                    parse=False, entity=False)
corpus = Corpus()
# Make a ranked list of rare vs frequent words
corpus.update_word_count(tokens)
corpus.finalize()
# The tokenization uses spaCy indices, and so may have gaps
# between indices for words that aren't present in our dataset.
# This builds a new compact index
compact = corpus.to_compact(tokens)
# Remove extremely rare words
pruned = corpus.filter_count(compact, min_count=50)
# Words tend to have power law frequency, so selectively
# downsample the most prevalent words
clean = corpus.subsample_frequent(pruned)
# Now flatten a 2D array of document per row and word position
# per column to a 1D array of words. This will also remove skips
# and OoV words
doc_ids = np.arange(pruned.shape[0])
flattened, (doc_ids,) = corpus.compact_to_flat(pruned, doc_ids)



# Model Parameters
# Number of documents
n_docs = len(texts)
# Number of unique words in the vocabulary
n_words = flattened.max() + 1
# Number of dimensions in a single word vector
n_hidden = 128
# Number of topics to fit
n_topics = 20
# Get the count for each key
counts = corpus.keys_counts[:n_words]
# Get the string representation for every compact key
words = corpus.word_list(vocab)[:n_words]

# Fit the model
model = LDA2Vec(n_words, n_hidden, counts, dropout_ratio=0.2)
model.add_categorical_feature(n_docs, n_topics, name='document_id')
model.finalize()

for _ in range(50):
    model.top_words_per_topic('document_id', words)
    if gpu:
        model.to_gpu()
    model.fit(flattened, categorical_features=[doc_ids], fraction=1e-3,
              epochs=1)
    model.to_cpu()
serializers.save_hdf5('model_film_20news.hdf5', model)

In [11]:
# Model Parameters
# Number of documents
n_docs = len(texts)
# Number of unique words in the vocabulary
n_words = flattened.max() + 1
# Number of dimensions in a single word vector
n_hidden = 128
# Number of topics to fit
n_topics = 20
# Get the count for each key
counts = corpus.keys_counts[:n_words]
# Get the string representation for every compact key
words = corpus.word_list(vocab)[:n_words]

# Fit the model
model = LDA2Vec(n_words, n_hidden, counts, dropout_ratio=0.2)
model.add_categorical_feature(n_docs, n_topics, name='document_id')
model.finalize()

for _ in range(50):
    model.top_words_per_topic('document_id', words)
    if gpu:
        model.to_gpu()
    model.fit(flattened, categorical_features=[doc_ids], fraction=1e-3,
              epochs=1)
    model.to_cpu()
serializers.save_hdf5('model_film_20news.hdf5', model)

INFO:root:Setup LDA2Vec class
INFO:root:Added categorical_feature document_id
INFO:root:Setup optimizer
INFO:root:Finalized the class
INFO:root:Partial fit loss: 3.63665e+04 Prior: -2.83307e+01
INFO:root:Partial fit loss: 3.44046e+04 Prior: -2.83308e+01
INFO:root:Partial fit loss: 3.26854e+04 Prior: -2.83308e+01
INFO:root:Partial fit loss: 3.82216e+04 Prior: -2.83308e+01
INFO:root:Partial fit loss: 3.59436e+04 Prior: -2.83308e+01
INFO:root:Partial fit loss: 3.34910e+04 Prior: -2.83309e+01
INFO:root:Partial fit loss: 3.46477e+04 Prior: -2.83309e+01
INFO:root:Partial fit loss: 3.34224e+04 Prior: -2.83310e+01
INFO:root:Partial fit loss: 3.43815e+04 Prior: -2.83310e+01
INFO:root:Partial fit loss: 3.37358e+04 Prior: -2.83311e+01
INFO:root:Partial fit loss: 3.26037e+04 Prior: -2.83311e+01
INFO:root:Partial fit loss: 3.15994e+04 Prior: -2.83312e+01
INFO:root:Partial fit loss: 3.21508e+04 Prior: -2.83312e+01
INFO:root:Partial fit loss: 3.31293e+04 Prior: -2.83313e+01
INFO:root:Partial fit loss

Top words in topic 0 u'want u'p u'posting out_of_vocabulary , u'l u'better u'bit ' u'thanks
Top words in topic 1 u'take out_of_vocabulary u'w u'think u'news u'problem u'power u'subject u'true u'year
Top words in topic 2 u'someone u'm u'right u'new u'even u'enough u'news u'mail u'n u'fact
Top words in topic 3 u'g u'things u'would u'reply u'time u'm u'said u'many [ u'u
Top words in topic 4 u'run u'w u'm u'must u'work u'even out_of_vocabulary u'edu u'v u'go
Top words in topic 5 u'believe u'right u'thanks u'usa u'world u'e u'see u'software u'ibm u'writes
Top words in topic 6 u'could u'part u'way u'david u'university u'go u'going u'run u'space u'lines
Top words in topic 7 u'know u'number u'r u'real u'first u'different u'case u'posting u'edu u'going
Top words in topic 8 u'please u'real u'world u'lines u'probably u'l , u'believe u've u'x
Top words in topic 9 u'know u'state u'someone u'new u'go u'true u'net u'm [ u've
Top words in topic 10 u'us u'sun u'real u'university u'different u'article u

INFO:root:Partial fit loss: 1.68595e+04 Prior: -2.96726e+01
INFO:root:Partial fit loss: 1.74242e+04 Prior: -2.96728e+01
INFO:root:Partial fit loss: 1.82237e+04 Prior: -2.96729e+01
INFO:root:Partial fit loss: 1.81931e+04 Prior: -2.96731e+01
INFO:root:Partial fit loss: 1.92054e+04 Prior: -2.96732e+01
INFO:root:Partial fit loss: 1.75356e+04 Prior: -2.96733e+01
INFO:root:Partial fit loss: 1.82083e+04 Prior: -2.96735e+01
INFO:root:Partial fit loss: 1.82304e+04 Prior: -2.96736e+01
INFO:root:Partial fit loss: 1.77520e+04 Prior: -2.96738e+01
INFO:root:Partial fit loss: 1.76178e+04 Prior: -2.96740e+01
INFO:root:Partial fit loss: 1.91108e+04 Prior: -2.96741e+01
INFO:root:Partial fit loss: 1.79797e+04 Prior: -2.96743e+01
INFO:root:Partial fit loss: 1.77309e+04 Prior: -2.96745e+01
INFO:root:Partial fit loss: 1.75625e+04 Prior: -2.96746e+01
INFO:root:Partial fit loss: 1.70024e+04 Prior: -2.96748e+01
INFO:root:Partial fit loss: 1.80519e+04 Prior: -2.96750e+01
INFO:root:Partial fit loss: 1.96306e+04 


Top words in topic 1 out_of_vocabulary u'news , u'u ' u'someone u'll u'make u'problem u'usa
Top words in topic 2 out_of_vocabulary , u'someone ' u'make u'news u'john u'u u'problem u'usa
Top words in topic 3 ' out_of_vocabulary u'u , u'someone u'news u'problem u'make u'john u'computer
Top words in topic 4 out_of_vocabulary u'john , u'someone out_of_vocabulary u'u u'news u'make ' u'usa
Top words in topic 5 out_of_vocabulary , u'usa u'problem ' u'make u'john u'u u'll u'someone
Top words in topic 6 , u'news out_of_vocabulary u'u u'problem u'someone ' out_of_vocabulary u'make u'usa
Top words in topic 7 out_of_vocabulary ' , u'news u'someone u'john u'll u'u u'usa out_of_vocabulary
Top words in topic 8 out_of_vocabulary , u'u u'someone u'news ' u'usa u'make u'll u'john
Top words in topic 9 out_of_vocabulary , u'news ' u'john out_of_vocabulary u'll u'someone u'u u'make
Top words in topic 10 out_of_vocabulary u'news , u'u ' u'someone u'john u'make u'll u'software
Top words in topic 11 out_of_v

INFO:root:Partial fit loss: 2.56063e+03 Prior: -2.97801e+01
INFO:root:Partial fit loss: 2.62436e+03 Prior: -2.97803e+01
INFO:root:Partial fit loss: 2.95880e+03 Prior: -2.97804e+01
INFO:root:Partial fit loss: 2.90644e+03 Prior: -2.97806e+01
INFO:root:Partial fit loss: 2.50938e+03 Prior: -2.97808e+01
INFO:root:Partial fit loss: 2.53563e+03 Prior: -2.97810e+01
INFO:root:Partial fit loss: 2.39499e+03 Prior: -2.97811e+01
INFO:root:Partial fit loss: 2.78145e+03 Prior: -2.97813e+01
INFO:root:Partial fit loss: 2.65238e+03 Prior: -2.97815e+01
INFO:root:Partial fit loss: 2.81872e+03 Prior: -2.97817e+01
INFO:root:Partial fit loss: 2.52066e+03 Prior: -2.97819e+01
INFO:root:Partial fit loss: 2.46722e+03 Prior: -2.97821e+01
INFO:root:Partial fit loss: 2.73982e+03 Prior: -2.97823e+01
INFO:root:Partial fit loss: 2.33122e+03 Prior: -2.97825e+01
INFO:root:Partial fit loss: 2.91438e+03 Prior: -2.97827e+01
INFO:root:Partial fit loss: 2.76394e+03 Prior: -2.97829e+01
INFO:root:Partial fit loss: 2.76949e+03 

...


Top words in topic 1 out_of_vocabulary u'true u'believe u'people u'god u'said u'used u'things u'two u've
Top words in topic 2 out_of_vocabulary u'cs u'windows u'university u'state u'computer u'technology u'ca u'help u'anyone
Top words in topic 3 out_of_vocabulary u'mark u'o u'computer u'someone u'host u'nntp u'com u'posting u'john
Top words in topic 4 out_of_vocabulary out_of_vocabulary u'uk u'used u'x u'software u'r u'something u'net u'use
Top words in topic 5 out_of_vocabulary u'uk u'mail u'ibm u'right u'technology u'org out_of_vocabulary u'reply u'distribution
Top words in topic 6 out_of_vocabulary u'ax u'com u'technology u'sun u'b u'r u'l u'david u'ibm
Top words in topic 7 out_of_vocabulary u'space u'two u'point u'go u'power u'course u'without u'believe u'god
Top words in topic 8 out_of_vocabulary u'o u'cc u'case u'u out_of_vocabulary u'edu u'power u'e u'reply
Top words in topic 9 out_of_vocabulary u'nasa u'space u'c u'r u'says u'news u'l u'article u'edu
Top words in topic 10 out_

INFO:root:Partial fit loss: 2.12598e+03 Prior: -6.09180e+01
INFO:root:Partial fit loss: 2.01482e+03 Prior: -6.09186e+01
INFO:root:Partial fit loss: 2.01461e+03 Prior: -6.09193e+01
INFO:root:Partial fit loss: 2.06170e+03 Prior: -6.09199e+01
INFO:root:Partial fit loss: 2.09124e+03 Prior: -6.09206e+01
INFO:root:Partial fit loss: 2.01276e+03 Prior: -6.09212e+01
INFO:root:Partial fit loss: 2.03648e+03 Prior: -6.09218e+01
INFO:root:Partial fit loss: 1.99993e+03 Prior: -6.09224e+01
INFO:root:Partial fit loss: 2.08678e+03 Prior: -6.09230e+01
INFO:root:Partial fit loss: 2.05913e+03 Prior: -6.09236e+01
INFO:root:Partial fit loss: 2.00811e+03 Prior: -6.09243e+01
INFO:root:Partial fit loss: 2.00026e+03 Prior: -6.09249e+01
INFO:root:Partial fit loss: 2.04940e+03 Prior: -6.09255e+01
INFO:root:Partial fit loss: 2.02668e+03 Prior: -6.09261e+01
INFO:root:Partial fit loss: 2.00081e+03 Prior: -6.09267e+01
INFO:root:Partial fit loss: 2.01209e+03 Prior: -6.09272e+01
INFO:root:Partial fit loss: 2.06659e+03 


Top words in topic 1 out_of_vocabulary u'true u'people u'believe u'god u'said u'things u'however u'used u'got
Top words in topic 2 out_of_vocabulary u'cs u'windows u'state u'university u'technology u'ca u'computer u'anyone u'new
Top words in topic 3 out_of_vocabulary u'mark u'o u'someone u'computer u'host u'john u'posting u'nntp u'com
Top words in topic 4 out_of_vocabulary out_of_vocabulary u'uk u'x u'used u'software u'r u'net u'something u'sun
Top words in topic 5 out_of_vocabulary u'uk u'right u'mail u'org u'technology u'ibm u'distribution u'reply out_of_vocabulary
Top words in topic 6 out_of_vocabulary u'com u'ax u'technology u'sun u'l u'r u'b u'david u'org
Top words in topic 7 out_of_vocabulary u'space u'point u'power u'two u'go u'even u'without u'different u'little
Top words in topic 8 out_of_vocabulary u'o u'cc u'u out_of_vocabulary u'case u'usa u'edu u'power u'please
Top words in topic 9 out_of_vocabulary u'nasa u'space u'c u'r u'says u'l u'news u'n u'article
Top words in topic

INFO:root:Partial fit loss: 2.00210e+03 Prior: -6.15505e+01
INFO:root:Partial fit loss: 2.02966e+03 Prior: -6.15512e+01
INFO:root:Partial fit loss: 2.03130e+03 Prior: -6.15519e+01
INFO:root:Partial fit loss: 2.08377e+03 Prior: -6.15525e+01
INFO:root:Partial fit loss: 1.96672e+03 Prior: -6.15532e+01
INFO:root:Partial fit loss: 2.03466e+03 Prior: -6.15538e+01
INFO:root:Partial fit loss: 1.97263e+03 Prior: -6.15545e+01
INFO:root:Partial fit loss: 1.91342e+03 Prior: -6.15551e+01
INFO:root:Partial fit loss: 2.06397e+03 Prior: -6.15558e+01
INFO:root:Partial fit loss: 1.95457e+03 Prior: -6.15565e+01
INFO:root:Partial fit loss: 2.05558e+03 Prior: -6.15572e+01
INFO:root:Partial fit loss: 1.94479e+03 Prior: -6.15578e+01
INFO:root:Partial fit loss: 2.02059e+03 Prior: -6.15585e+01
INFO:root:Partial fit loss: 1.98516e+03 Prior: -6.15592e+01
INFO:root:Partial fit loss: 2.19514e+03 Prior: -6.15598e+01
INFO:root:Partial fit loss: 1.99840e+03 Prior: -6.15604e+01
INFO:root:Partial fit loss: 1.95497e+03 


Top words in topic 1 out_of_vocabulary u'true u'people u'believe u'got u'said u'things u'different u'god u've
Top words in topic 2 u'cs out_of_vocabulary u'windows u'university u'state u'ca u'computer u'technology u'help u'new
Top words in topic 3 out_of_vocabulary u'mark u'o u'someone u'computer u'com u'john u'host u'posting u'nntp
Top words in topic 4 out_of_vocabulary out_of_vocabulary u'uk u'x u'used u'software u'r u'sun u'something u'com
Top words in topic 5 out_of_vocabulary u'uk u'right u'mail u'org u'ibm u'reply u'distribution u'g u'technology
Top words in topic 6 out_of_vocabulary u'ax u'com u'sun u'r u'b u'technology u'david u'ibm u'l
Top words in topic 7 out_of_vocabulary u'space u'point u'course u'two u'different u'power u'even u'go u'without
Top words in topic 8 out_of_vocabulary u'o u'cc u'case u'u out_of_vocabulary u'usa u'university u'edu u'please
Top words in topic 9 out_of_vocabulary u'nasa u'space u'r u'c u'says u'l u'article u'new u'news
Top words in topic 10 out_o